In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from rfpimp import importances
%matplotlib inline
pd.set_option("display.max_columns", 2000)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [184]:
df_full=pd.read_csv('data/Seasons_Stats.csv')
df_full.drop(columns='Unnamed: 0', inplace=True)

In [185]:
# df_full=df_full[df_full['Year']!=np.nan]

In [186]:
df_full.drop(labels='blank2', axis=1, inplace=True)
df_full.drop(labels='blanl', axis=1, inplace=True)
df=df_full[df_full['Year']>=1980]
df['Year']=df['Year'].astype(int)

df['Year']=pd.to_datetime(df['Year'], format='%Y')
df['Year']=df['Year'].dt.year

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs

In [187]:
df_coach=pd.read_csv('data/coach_data_cleaned.csv')

In [188]:
df_coach.drop(labels=['Unnamed: 0'],axis=1,inplace=True)

In [189]:
df_coach=df_coach[df_coach['season']!='Career']
df_coach['season']=df_coach['season'].apply(lambda x: x[:2]+x[-2:]) #code for fixing ther dates

In [190]:
df_coach['season']=df_coach['season'].astype(int)

In [191]:
df_coach=df_coach[df_coach['season']>=1980]

In [192]:
df_coach[df_coach['season']==1980]

,Team,coach_age,coach_name,season,playoff_games,playoff_wins,playoff_loses,playoff_W/L%,season_games,season_wins,season_loses,season_W/L%,season_position
24,DET,42.0,Richie Adubato,1980,NaN,NaN,NaN,NaN,70,12.0,58.0,0.171,6.0
37,CLE,48.0,Stan Albeck,1980,NaN,NaN,NaN,NaN,82,37.0,45.0,0.451,5.0
61,GSW,43.0,Al Attles,1980,NaN,NaN,NaN,NaN,61,18.0,43.0,0.295,6.0
89,GSW,55.0,Johnny Bach,1980,NaN,NaN,NaN,NaN,21,6.0,15.0,0.286,6.0
108,SAS,51.0,Bob Bass,1980,3.0,1.0,2.0,0.333,16,8.0,8.0,0.500,3.0
228,ATL,46.0,Hubie Brown,1980,5.0,1.0,4.0,0.200,82,50.0,32.0,0.610,1.0
474,PHI,36.0,Billy Cunningham,1980,18.0,12.0,6.0,0.667,82,59.0,23.0,0.720,2.0
611,BOS,45.0,Bill Fitch,1980,9.0,5.0,4.0,0.556,82,61.0,21.0,0.744,1.0
638,KCK,48.0,Cotton Fitzsimmons,1980,3.0,1.0,2.0,0.333,82,47.0,35.0,0.573,2.0
782,HOU,42.0,Del Harris,1980,7.0,2.0,5.0,0.286,82,41.0,41.0,0.500,2.0


In [193]:
df_coach.columns.values

array(['Team', 'coach_age', 'coach_name', 'season', 'playoff_games',
       'playoff_wins', 'playoff_loses', 'playoff_W/L%', 'season_games',
       'season_wins', 'season_loses', 'season_W/L%', 'season_position'],
      dtype=object)

In [194]:
cols=['Team', 'coach_age', 'coach_name', 'season','season_games',
       'season_wins', 'season_loses', 'season_W/L%', 'season_position']

In [195]:
df_coach=df_coach[cols]

In [196]:
df_coach.head()

,Team,coach_age,coach_name,season,season_games,season_wins,season_loses,season_W/L%,season_position
0,POR,42.0,Rick Adelman,1989,35,14.0,21.0,0.400,5.0
1,POR,43.0,Rick Adelman,1990,82,59.0,23.0,0.720,2.0
2,POR,44.0,Rick Adelman,1991,82,63.0,19.0,0.768,1.0
3,POR,45.0,Rick Adelman,1992,82,57.0,25.0,0.695,1.0
4,POR,46.0,Rick Adelman,1993,82,51.0,31.0,0.622,3.0


In [197]:
df.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
5727,1980,Kareem Abdul-Jabbar*,C,32.0,LAL,82.0,NaN,3143.0,25.3,0.639,0.001,0.344,7.2,22.2,15.4,16.5,1.2,4.6,15.7,24.1,9.5,5.3,14.8,0.227,4.0,2.7,6.7,6.8,835.0,1383.0,0.604,0.0,1.0,0.000,835.0,1382.0,0.604,0.604,364.0,476.0,0.765,190.0,696.0,886.0,371.0,81.0,280.0,297.0,216.0,2034.0
5728,1980,Tom Abernethy,PF,25.0,GSW,67.0,NaN,1222.0,11.0,0.511,0.003,0.258,5.4,12.0,8.6,9.3,1.4,0.6,9.9,13.3,1.2,0.8,2.0,0.080,-1.5,-0.1,-1.6,0.1,153.0,318.0,0.481,0.0,1.0,0.000,153.0,317.0,0.483,0.481,56.0,82.0,0.683,62.0,129.0,191.0,87.0,35.0,12.0,39.0,118.0,362.0
5729,1980,Alvan Adams,C,25.0,PHO,75.0,NaN,2168.0,19.2,0.571,0.002,0.270,8.2,22.4,15.4,21.6,2.3,1.4,18.2,21.9,3.1,3.9,7.0,0.155,1.6,2.8,4.4,3.5,465.0,875.0,0.531,0.0,2.0,0.000,465.0,873.0,0.533,0.531,188.0,236.0,0.797,158.0,451.0,609.0,322.0,108.0,55.0,218.0,237.0,1118.0
5730,1980,Tiny Archibald*,PG,31.0,BOS,80.0,80.0,2864.0,15.3,0.574,0.023,0.548,2.3,5.3,3.8,30.2,1.7,0.2,19.7,17.0,5.9,2.9,8.9,0.148,1.1,-1.1,0.0,1.5,383.0,794.0,0.482,4.0,18.0,0.222,379.0,776.0,0.488,0.485,361.0,435.0,0.830,59.0,138.0,197.0,671.0,106.0,10.0,242.0,218.0,1131.0
5731,1980,Dennis Awtrey,C,31.0,CHI,26.0,NaN,560.0,7.4,0.524,0.000,0.833,6.0,16.9,11.5,9.0,1.0,1.5,24.8,7.9,0.1,0.5,0.6,0.053,-2.9,1.5,-1.4,0.1,27.0,60.0,0.450,0.0,0.0,NaN,27.0,60.0,0.450,0.450,32.0,50.0,0.640,29.0,86.0,115.0,40.0,12.0,15.0,27.0,66.0,86.0


In [221]:
temp=df_coach[df_coach.duplicated(subset=['Team','season'])]

In [ ]:
temp.sort(columns=[''])

In [201]:
# merging the tables:
result=pd.merge(df,df_coach, how='inner', left_on=['Year','Tm'], right_on=['season','Team'])

In [202]:
result.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Team,coach_age,coach_name,season,season_games,season_wins,season_loses,season_W/L%,season_position
0,1980,Kareem Abdul-Jabbar*,C,32.0,LAL,82.0,NaN,3143.0,25.3,0.639,0.001,0.344,7.2,22.2,15.4,16.5,1.2,4.6,15.7,24.1,9.5,5.3,14.8,0.227,4.0,2.7,6.7,6.8,835.0,1383.0,0.604,0.0,1.0,0.0,835.0,1382.0,0.604,0.604,364.0,476.0,0.765,190.0,696.0,886.0,371.0,81.0,280.0,297.0,216.0,2034.0,LAL,44.0,Jack McKinney,1980,14,10.0,4.0,0.714,1.0
1,1980,Kareem Abdul-Jabbar*,C,32.0,LAL,82.0,NaN,3143.0,25.3,0.639,0.001,0.344,7.2,22.2,15.4,16.5,1.2,4.6,15.7,24.1,9.5,5.3,14.8,0.227,4.0,2.7,6.7,6.8,835.0,1383.0,0.604,0.0,1.0,0.0,835.0,1382.0,0.604,0.604,364.0,476.0,0.765,190.0,696.0,886.0,371.0,81.0,280.0,297.0,216.0,2034.0,LAL,40.0,Paul Westhead,1980,68,50.0,18.0,0.735,1.0
2,1980,Ron Boone,SG,33.0,LAL,6.0,NaN,106.0,3.0,0.395,0.000,0.175,4.5,6.6,5.7,7.8,2.2,0.0,23.2,21.2,-0.3,0.1,-0.2,-0.098,-7.5,-1.8,-9.2,-0.2,14.0,40.0,0.350,0.0,0.0,NaN,14.0,40.0,0.350,0.350,6.0,7.0,0.857,4.0,7.0,11.0,7.0,5.0,0.0,13.0,13.0,34.0,LAL,44.0,Jack McKinney,1980,14,10.0,4.0,0.714,1.0
3,1980,Ron Boone,SG,33.0,LAL,6.0,NaN,106.0,3.0,0.395,0.000,0.175,4.5,6.6,5.7,7.8,2.2,0.0,23.2,21.2,-0.3,0.1,-0.2,-0.098,-7.5,-1.8,-9.2,-0.2,14.0,40.0,0.350,0.0,0.0,NaN,14.0,40.0,0.350,0.350,6.0,7.0,0.857,4.0,7.0,11.0,7.0,5.0,0.0,13.0,13.0,34.0,LAL,40.0,Paul Westhead,1980,68,50.0,18.0,0.735,1.0
4,1980,Marty Byrnes,SF,23.0,LAL,32.0,NaN,194.0,6.3,0.557,0.000,0.300,5.5,9.3,7.6,7.9,1.2,0.3,28.0,16.2,-0.1,0.2,0.0,0.009,-4.3,-1.7,-6.0,-0.2,25.0,50.0,0.500,0.0,0.0,NaN,25.0,50.0,0.500,0.500,13.0,15.0,0.867,9.0,18.0,27.0,13.0,5.0,1.0,22.0,32.0,63.0,LAL,44.0,Jack McKinney,1980,14,10.0,4.0,0.714,1.0


In [175]:
result.columns.values
cols=['Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'PER',
       'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM',
       'DBPM', 'BPM', 'VORP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Team',
       'coach_age', 'coach_name','season_games', 'season_wins',
       'season_loses', 'season_W/L%', 'season_position']

In [176]:
tot=result[cols]

In [177]:
tot.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Team,coach_age,coach_name,season_games,season_wins,season_loses,season_W/L%,season_position
0,1980,Kareem Abdul-Jabbar*,C,32.0,LAL,82.0,NaN,3143.0,25.3,0.639,0.001,0.344,7.2,22.2,15.4,16.5,1.2,4.6,15.7,24.1,9.5,5.3,14.8,0.227,4.0,2.7,6.7,6.8,835.0,1383.0,0.604,0.0,1.0,0.0,835.0,1382.0,0.604,0.604,364.0,476.0,0.765,190.0,696.0,886.0,371.0,81.0,280.0,297.0,216.0,2034.0,LAL,44.0,Jack McKinney,14.0,10.0,4.0,0.714,1.0
1,1980,Kareem Abdul-Jabbar*,C,32.0,LAL,82.0,NaN,3143.0,25.3,0.639,0.001,0.344,7.2,22.2,15.4,16.5,1.2,4.6,15.7,24.1,9.5,5.3,14.8,0.227,4.0,2.7,6.7,6.8,835.0,1383.0,0.604,0.0,1.0,0.0,835.0,1382.0,0.604,0.604,364.0,476.0,0.765,190.0,696.0,886.0,371.0,81.0,280.0,297.0,216.0,2034.0,LAL,40.0,Paul Westhead,68.0,50.0,18.0,0.735,1.0
2,1980,Tom Abernethy,PF,25.0,GSW,67.0,NaN,1222.0,11.0,0.511,0.003,0.258,5.4,12.0,8.6,9.3,1.4,0.6,9.9,13.3,1.2,0.8,2.0,0.080,-1.5,-0.1,-1.6,0.1,153.0,318.0,0.481,0.0,1.0,0.0,153.0,317.0,0.483,0.481,56.0,82.0,0.683,62.0,129.0,191.0,87.0,35.0,12.0,39.0,118.0,362.0,GSW,43.0,Al Attles,61.0,18.0,43.0,0.295,6.0
3,1980,Tom Abernethy,PF,25.0,GSW,67.0,NaN,1222.0,11.0,0.511,0.003,0.258,5.4,12.0,8.6,9.3,1.4,0.6,9.9,13.3,1.2,0.8,2.0,0.080,-1.5,-0.1,-1.6,0.1,153.0,318.0,0.481,0.0,1.0,0.0,153.0,317.0,0.483,0.481,56.0,82.0,0.683,62.0,129.0,191.0,87.0,35.0,12.0,39.0,118.0,362.0,GSW,55.0,Johnny Bach,21.0,6.0,15.0,0.286,6.0
4,1980,Alvan Adams,C,25.0,PHO,75.0,NaN,2168.0,19.2,0.571,0.002,0.270,8.2,22.4,15.4,21.6,2.3,1.4,18.2,21.9,3.1,3.9,7.0,0.155,1.6,2.8,4.4,3.5,465.0,875.0,0.531,0.0,2.0,0.0,465.0,873.0,0.533,0.531,188.0,236.0,0.797,158.0,451.0,609.0,322.0,108.0,55.0,218.0,237.0,1118.0,PHO,42.0,John MacLeod,82.0,55.0,27.0,0.671,3.0


In [178]:
tot['prev_year']=tot['Year']-1

In [183]:
tot.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Team,coach_age,coach_name,season_games,season_wins,season_loses,season_W/L%,season_position,prev_year
0,1980,Kareem Abdul-Jabbar*,C,32.0,LAL,82.0,NaN,3143.0,25.3,0.639,0.001,0.344,7.2,22.2,15.4,16.5,1.2,4.6,15.7,24.1,9.5,5.3,14.8,0.227,4.0,2.7,6.7,6.8,835.0,1383.0,0.604,0.0,1.0,0.0,835.0,1382.0,0.604,0.604,364.0,476.0,0.765,190.0,696.0,886.0,371.0,81.0,280.0,297.0,216.0,2034.0,LAL,44.0,Jack McKinney,14.0,10.0,4.0,0.714,1.0,1979
1,1980,Kareem Abdul-Jabbar*,C,32.0,LAL,82.0,NaN,3143.0,25.3,0.639,0.001,0.344,7.2,22.2,15.4,16.5,1.2,4.6,15.7,24.1,9.5,5.3,14.8,0.227,4.0,2.7,6.7,6.8,835.0,1383.0,0.604,0.0,1.0,0.0,835.0,1382.0,0.604,0.604,364.0,476.0,0.765,190.0,696.0,886.0,371.0,81.0,280.0,297.0,216.0,2034.0,LAL,40.0,Paul Westhead,68.0,50.0,18.0,0.735,1.0,1979
2,1980,Tom Abernethy,PF,25.0,GSW,67.0,NaN,1222.0,11.0,0.511,0.003,0.258,5.4,12.0,8.6,9.3,1.4,0.6,9.9,13.3,1.2,0.8,2.0,0.080,-1.5,-0.1,-1.6,0.1,153.0,318.0,0.481,0.0,1.0,0.0,153.0,317.0,0.483,0.481,56.0,82.0,0.683,62.0,129.0,191.0,87.0,35.0,12.0,39.0,118.0,362.0,GSW,43.0,Al Attles,61.0,18.0,43.0,0.295,6.0,1979
3,1980,Tom Abernethy,PF,25.0,GSW,67.0,NaN,1222.0,11.0,0.511,0.003,0.258,5.4,12.0,8.6,9.3,1.4,0.6,9.9,13.3,1.2,0.8,2.0,0.080,-1.5,-0.1,-1.6,0.1,153.0,318.0,0.481,0.0,1.0,0.0,153.0,317.0,0.483,0.481,56.0,82.0,0.683,62.0,129.0,191.0,87.0,35.0,12.0,39.0,118.0,362.0,GSW,55.0,Johnny Bach,21.0,6.0,15.0,0.286,6.0,1979
4,1980,Alvan Adams,C,25.0,PHO,75.0,NaN,2168.0,19.2,0.571,0.002,0.270,8.2,22.4,15.4,21.6,2.3,1.4,18.2,21.9,3.1,3.9,7.0,0.155,1.6,2.8,4.4,3.5,465.0,875.0,0.531,0.0,2.0,0.0,465.0,873.0,0.533,0.531,188.0,236.0,0.797,158.0,451.0,609.0,322.0,108.0,55.0,218.0,237.0,1118.0,PHO,42.0,John MacLeod,82.0,55.0,27.0,0.671,3.0,1979


In [182]:
tot=tot.drop_duplicates()

In [181]:
pd.merge(tot,tot[['Year','Tm','coach_name']],how='left', left_on=['Tm','prev_year'], right_on=['Tm'])

ValueError: can not merge DataFrame with instance of type <class 'pandas.core.series.Series'>